### Get results from Wandb

In [1]:
import pandas as pd 
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("draftrec/atari_pretrain")

summary_list, config_list, id_list = [], [], []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    id_list.append(run.id)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "id": id_list,
    })

runs_df.to_csv("project.csv")

### Read runs

In [2]:
data_ = pd.read_csv('project.csv', index_col=0)
data_

,summary,config,id
0,{'_wandb': {'runtime': 1}},"{'exp_name': 'trajformer_vid_cont', 'overrides...",3daftxps
1,"{'neg_sim': 0.00020965789881077513, 'pos_sim':...","{'env': {'game': 'Hero', 'type': 'atari', 'fra...",lj4cmmsc
2,"{'loss': 0.11008913078159094, 'epoch': 10, '_r...","{'env': {'game': 'Jamesbond', 'type': 'atari',...",1himt4yy
3,"{'_step': 42000, '_runtime': 20191, 'pos_neg_d...","{'env': {'game': 'Frostbite', 'type': 'atari',...",1yehqmxm
4,"{'act_f1': 0.2147300639483379, 'mean_grad_norm...","{'env': {'game': 'Gopher', 'type': 'atari', 'f...",xopaukj1
...,...,...,...
748,"{'_wandb': {'runtime': 20220}, '_runtime': 201...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",5zlpym7v
749,"{'positive_sim': 0.998840137720108, 'loss': -0...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",3oh4f4sw
750,"{'_runtime': 18375, '_timestamp': 1659280529, ...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",385p3sw7
751,"{'loss': -0.9872578982114792, '_step': 117001,...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",njf5gwab


### Filter based on conditions

In [3]:
group_name = 'baseline_10'
exp_name = 'trajformer_vid_cont'
model_path = '0/10/model.pth'

In [4]:
configs = data_['config']
indexs = []
for idx, config in enumerate(configs):
    cfg = eval(config)
    if len(cfg) == 0:
        continue
        
    run_exp_name = cfg['exp_name']
    run_group_name = cfg['group_name']

    # condition
    if run_exp_name == exp_name and run_group_name == group_name:
        if 'env' in cfg:
            indexs.append(idx)

In [5]:
data = data_.iloc[indexs]
envs = []
for config in data['config']:
    envs.append(eval(config)['env']['game'])
data['env'] = envs
print(len(data))
data

26


/tmp/ipykernel_9253/3806693401.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['env'] = envs


,summary,config,id,env
13,"{'epoch': 10, 'act_f1': 0.36188711405702234, '...","{'env': {'game': 'UpNDown', 'type': 'atari', '...",283zu7t4,UpNDown
14,"{'obs_loss': 0.8202280444181386, 'reward_f1': ...","{'env': {'game': 'Seaquest', 'type': 'atari', ...",1amhofku,Seaquest
57,"{'reward_ratio': 0.0032501625081254064, 'min_g...","{'env': {'game': 'PrivateEye', 'type': 'atari'...",2s0eqs8w,PrivateEye
58,"{'_wandb': {'runtime': 51328}, 'act_f1': 0.375...","{'env': {'game': 'RoadRunner', 'type': 'atari'...",1gay5kzk,RoadRunner
59,"{'_step': 234370, 'reward_f1': 0.9118220834959...","{'env': {'game': 'Qbert', 'type': 'atari', 'fr...",3iyjeqa6,Qbert
60,"{'act_f1': 0.394826304879973, 'pos_neg_diff': ...","{'env': {'game': 'Pong', 'type': 'atari', 'fra...",31c7ofut,Pong
75,"{'pos_neg_diff': 0.5475814634868446, 'reward_r...","{'env': {'game': 'KungFuMaster', 'type': 'atar...",138jpfc9,KungFuMaster
76,"{'reward_ratio': 0.1559577978898945, 'epoch': ...","{'env': {'game': 'MsPacman', 'type': 'atari', ...",927z1e6n,MsPacman
77,"{'_runtime': 51224, 'act_loss': 0, 'idm_loss':...","{'env': {'game': 'Krull', 'type': 'atari', 'fr...",38osraxc,Krull
78,"{'loss': 1.100215765518871, '_step': 234370, '...","{'env': {'game': 'Kangaroo', 'type': 'atari', ...",385wbofy,Kangaroo


### Restore Saved Models

In [6]:
import pathlib
base_path = str(pathlib.Path().resolve())

artifact_dict = {}
for run_id, env in zip(data['id'], data['env']):
    print(env, run_id)
    try:
        name = env + '/' + model_path 
        path = base_path + '/' + name
        wandb.restore(name, run_path="draftrec/atari_pretrain/" + run_id)
        artifact_dict[path] = name 
    except:
        continue

UpNDown 283zu7t4
Seaquest 1amhofku
PrivateEye 2s0eqs8w
RoadRunner 1gay5kzk
Qbert 3iyjeqa6
Pong 31c7ofut
KungFuMaster 138jpfc9
MsPacman 927z1e6n
Krull 38osraxc
Kangaroo 385wbofy
Frostbite 2ppxiack
Jamesbond cu3vflx8
Hero 1y4dvqmd
Gopher 9g40j2e6
Freeway 1se2l9i3
ChopperCommand 3gqom1ct
DemonAttack 19qxcnvn
CrazyClimber r07370h5
BankHeist 11ydjikf
BattleZone 2nh5h134
Breakout 351837xe
Boxing 12rdde80
Asterix 3evfr7pl
Amidar irx0j53b
Alien 3aelhdv4
Assault 2mpkmvjm


### Save to artifact

In [7]:
wandb.init(project='atari_pretrain',
           entity='draftrec',
           group=exp_name,
           settings=wandb.Settings(start_method="thread"))  
artifact = wandb.Artifact(name=exp_name, type='model')

# save models
for path, name in artifact_dict.items():
    artifact.add_file(path, name=name)

wandb.run.finish_artifact(artifact)

wandb: Currently logged in as: joonleesky (draftrec). Use `wandb login --relogin` to force relogin
/opt/conda/lib/python3.8/site-packages/IPython/html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


### Remove artifact

In [8]:
import shutil
for env in data['env']:
    shutil.rmtree('./'+env, ignore_errors=True)

### Generate json file

In [17]:
def get_results_dict(data):
    results = []
    for idx in range(len(data)):
        row = data.iloc[idx]
        summary = eval(row['summary'])
        config = eval(row['config'])

        if 'env' not in config:
            continue

        game = config['env']['game']
        try:
            score = summary['eval_mean_traj_game_scores']
        except:
            continue
        results.append([0, game, score, 0])

    return results
    
results = get_results_dict(data)

In [18]:
import json

def generate_json_file(file_name, results):
    data = {}
    data['data'] = results
    path = file_name + '.json'
    with open(path, 'w') as json_file:
        json.dump(data, json_file)

In [19]:
generate_json_file('byol_impala', results)